# LightGBM 7th place solution

참고 : https://www.kaggle.com/jsaguiar/lightgbm-7th-place-solution

이 모델은 appplication의 범주형 변수에 goss와 label encode를 사용한 LightGBM을 사용한다.

In [1]:
import os
import gc
import time
import numpy as np
import pandas as pd
from contextlib import contextmanager
import multiprocessing as mp
from functools import partial
from scipy.stats import kurtosis, iqr, skew
from lightgbm import LGBMClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)

In [2]:
def main(debug=False):
    num_rows = 30000 if debug else None
    
    with timer('application_train and application_test'):
        df = get_train_test(DATA_DIRECTORY, num_rows=num_rows)
        print('Application dataframe shape :', df.shape)
        
    with timer('Bureau and bureau_balance data'):
        bureau_df = get_bureau(DATA_DIRECTORY, num_rows=num_rows)
        print('Bureau dataframe shape :', bureau.shape)
        del bureau_df
        gc.collet()
        
    with timer('previous_application'):
        prev_df = get_previous_applications(DATA_DIRECTORY, num_rows)
        df = pd.merge(df, prev_df, on='SK_ID_CURR', how='left')
        print('Previous dataframe shape :', prev_df.shape)
        del prev_df
        gc.collect()
        
    with timer('previous applications balances'):
        pos = get_pos_cash(DATA_DIRECTORY, num_rows)
        df = pd.merge(df, pos, on='SK_ID_CURR', how='left')
        print('Pos=cash dataframe shape :', pos.shape)
        del pos
        gc.collet()
        
        ins = get_installment_payments(DAYA_DIRECTORY, num_rows)
        df = pd.merge(df, ins, on='SK_ID_CURR', how='left')
        print('Installments dataframe shape :', ins.shape)
        del ins
        gc.collect()
        
        cc = get_credit_card(DATA_DIRECTORY, num_rows)
        df = pd.merge(df, cc, on='SK_ID_CURR', how='left')
        print('Credit card dataframe shape :', cc.shape)
        del cc
        gc.collect()
        
    # 비율 추가 및 다른 테이블간 그룹화
    df = add_ratios_features(df)
    df = reduce_memory(df)
    lgbm_categorical_feat = ['CODE_GENDER', 'FLAG_OWN_CAR', 'NAME_CONTRACT_TYPE', 'NAME_EDUCATION_TYPE',
        'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'NAME_INCOME_TYPE', 'OCCUPATION_TYPE',
        'ORGANIZATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_TYPE_SUITE', 'WALLSMATERIAL_MODE']
    with timer('Run LightGBM'):
        feat_importance = kfold_lightgbm_sklearn(df, lgbm_categorical_feat)
        print(feat_importance)

In [3]:
# 비율 추가 함수
def add_ratios_features(df):
    # 신용 대 소득 비율
    df['BUREAU_INCOME_CREDIT_RATIO'] = df['BUREAU_AMT_CREDIT_SUM_MEAN'] / df['AMT_INCOME_TOTAL']
    df['BUREAU_ACTIVE_CREDIT_TO_INCOME_RATIO'] = df['BUREAU_ACTIVE_AMT_CREDIT_SUM_SUM'] / df['AMT_INCOME_TOTAL']
    # 이전 신용 대 현재 신용 비율
    df['CURRENT_TO_APPROVED_CREDIT_MIN_RATIO'] = df['APPROVED_AMT_CREDIT_MIN'] / df['AMT_CREDIT']
    df['CURRENT_TO_APPROVED_CREDIT_MAX_RATIO'] = df['APPROVED_AMT_CREDIT_MAX'] / df['AMT_CREDIT']
    df['CURRENT_TO_APPROVED_CREDIT_MEAN_RATIO'] = df['APPROVED_AMT_CREDIT_MEAN'] / df['AMT_CREDIT']
    # 이전 신용 대 현재 연금 비율
    df['CURRENT_TO_APPROVED_ANNUITY_MAX_RATIO'] = df['APPROVED_AMT_ANNUITY_MAX'] / df['AMT_ANNUITY']
    df['CURRENT_TO_APPROVED_ANNUITY_MEAN_RATIO'] = df['APPROVED_AMT_ANNUITY_MEAN'] / df['AMT_ANNUITY']
    df['PAYMENT_MIN_TO_ANNUITY_RATIO'] = df['INS_AMT_PAYMENT_MIN'] / df['AMT_ANNUITY']
    df['PAYMENT_MAX_TO_ANNUITY_RATIO'] = df['INS_AMT_PAYMENT_MAX'] / df['AMT_ANNUITY']
    df['PAYMENT_MEAN_TO_ANNUITY_RATIO'] = df['INS_AMT_PAYMENT_MEAN'] / df['AMT_ANNUITY']
    # 이전신용 대 현재신용 대 연금 비율
    df['CTA_CREDIT_TO_ANNUITY_MAX_RATIO'] = df['APPROVED_CREDIT_TO_ANNUITY_RATIO_MAX'] / df[
        'CREDIT_TO_ANNUITY_RATIO']
    df['CTA_CREDIT_TO_ANNUITY_MEAN_RATIO'] = df['APPROVED_CREDIT_TO_ANNUITY_RATIO_MEAN'] / df[
        'CREDIT_TO_ANNUITY_RATIO']
    # 일수 차이 및 비율
    df['DAYS_DECISION_MEAN_TO_BIRTH'] = df['APPROVED_DAYS_DECISION_MEAN'] / df['DAYS_BIRTH']
    df['DAYS_CREDIT_MEAN_TO_BIRTH'] = df['BUREAU_DAYS_CREDIT_MEAN'] / df['DAYS_BIRTH']
    df['DAYS_DECISION_MEAN_TO_EMPLOYED'] = df['APPROVED_DAYS_DECISION_MEAN'] / df['DAYS_EMPLOYED']
    df['DAYS_CREDIT_MEAN_TO_EMPLOYED'] = df['BUREAU_DAYS_CREDIT_MEAN'] / df['DAYS_EMPLOYED']
    
    return df

## LgithGBM Model

In [4]:
def kfold_lightgbm_sklearn(data, categorical_feature=None):
    df = data[data['TARGET'].notnull()]  # TARGET이 결측값이 아닌 데이터
    test = data[data['TARGET'].isnull()]  # TARGET이 결측값인 데이터 (test는 target없음)
    print('Train/valid shape : {}, test shape : {}'.format(df.shape,test.shape))
    
    del_features = ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index','levle_0']
    predictors = list(filter(lambda v: v not in del_features, df.columns))  # 삭제할 변수가 아닌 변수 리스트
    
    if not STRATIFIED_KFOLD:
        folds = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=RANDOM_SEED)
    else:
        folds = StratifiedKFold(n_splits=NUM_FOLDS,shuffle=True,random_state=RANDOM_SEED)
        
    
    # oof 예측, test 예측, 변수 중요도, train/valid auc
    oof_preds = np.zeros(df.shape[0])
    sub_preds = np.zeros(test.shape[0])
    importance_df = pd.DataFrame()
    eval_results = dict()
    
    for n_fold, (train_idx,valid_idx) in enumerate(folds.split(df[predictors],df['TARGET'])):
        train_x, train_y = df[predictors].iloc[train_idx], df['TARGET'].iloc[train_idx]
        valid_x, valid_y = df[predictors].iloc[valid_idx], df['TARGET'].iloc[valid_idx]
        
        params = {'random_state':RANDOM_SEED, 'nthread':NUM_THREADS}
        clf = LGBMClassifier(**{**params, **LIGHTGBM_PARMAS})
        if not categorical_feature:  ### 범주형 변수 아니면
            clf.fit(train_x,train_y, eval_set=[(train_x,train_y),(valid_x,valid_y)],
                   eval_metric='auc',verbose=400,early_stopping_rounds=EARLY_STOPPING)
        else :  ### 범주형 변수면 맨 아래줄 추가
            clf.fit(train_x,train_y, eval_set=[(train_x,train_y),(valid_X,valid_y)],
                   eval_metric='auc',verbose=400,early_stopping_rounds=EARLY_STTOPING,  
                   feature_name=list(df[predictors].columns),categorical_feautre=categorical_feature)
        
        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:,1]
        sub_preds += clf.predict_proba(test[predictors], num_iteration=clf.best_iteration_)[:,1] / folds.n_splits
        
        # GAIN, SPLIT 사용한 변수 중요도
        fold_importance = pd.DataFrame()
        fold_importance['feature'] = predictors
        fold_importance['gain'] = clf.booster_.feature_importance(importance_type='gain')
        fold_importance['split'] = clf.booster_.feature_importance(importance_type='split')
        importance_df = pd.concat([importance_df, fold-importance],axis=0)
        eval_results['train_{}'.format(n_fold+1)] = clf.evals_result_['training']['auc']
        eval_results['valid_{}'.format(n_fold+1)] = clf.evals_result_['valid_1']['auc']
        
        print('Fold %2d AUC : %.6f' %(n_fold+1, roc_auc_score(valid_y,oof_preds[valid_idx])))
        del clf, train_x, tarin_y, valid_x, valid_y
        gc.collect()
        
    print('Full AUC score %.6f' %roc_auc_score(df['TARGET'],oof_preds))
    test['TARGET'] = sub_preds.copy()
    
    # folds간 평균 변수 중요도
    mean_importance = importance_df.groupby('feature').mean().reset_index()
    mean_importance.sort_values(by='gain', ascending=False, inplace=True)
    
    # 변수중요도, test 예측, oof 예측을 csv로 저장
    if GENERATE_SUBMISSION_FILES:
        # oof csv
        oof = pd.DataFrame()
        oof['SK_ID_CURR'] = df['SK_ID_CURR'].copy()
        df['PREDICTIONS'] = oof_preds.copy()
        df['TARGET'] = df['TARGET'].copy()
        df.to_csv('oof{}.csv'.format(SUBMISSION_SUFIX),index=False)
        # 제출(test)파일과 변수중요도 저장
        test[['SK_ID_CURR','TARGET']].to_csv('submission{}.csv'.format(SUBMISSION_SUFIX),index=False)
        mean_importance.to_csv('feature_importance{}.csv'.format(SUBMISSION_SUFIX),index=False)
        
    return mean_importance

## Application Pipeline

In [6]:
def get_train_test(path, num_rows=None):  # application_train/test.csv 처리 및 데이터프레임 반환
    train = pd.read_csv(os.path.join(path,'application_train.csv'),nrows=num_rows)
    test = pd.read_csv(os.path.join(path,'application_test.csv'),nrows=num_rows)
    df = train.append(test)
    del train,test
    gc.collect()
    
    # Data cleaning
    df = df[df['CODE_GENDER']!='XNA']  # XNA 코드 성별 가진 사람 4명
    df = df[df['AMT_INCOME_TOTAL']<20000000]  # test 최대 income 4M, train 117M
    df['DAYS_EMPLOYED'].replace(365243,np.nan,inplace=True)  ### 이상치인 365243을 nan으로 대체
    df['DAYS_LAST_PHONE_CHANGE'].replace(0,np.nan,inplace=True)
    
    # Flag_document 변수 - 개수 및 첨도
    docs = [f for f in df.columns if 'FLAG_DOC' in f]  # Flag_document 변수 저장
    df['DOCUMETN_COUNT'] = df[docs].sum(axis=1)
    df['NEW_DOC_KURT'] = df[docs].kurtosis(axis=1)
    # 범주형 변수 age - target=1인 plot
    df['AGE_RANGE'] = df['DAYS_BIRTH'].apply(lambda x:get_age_label(x))
    
    # External source 변수 통한 새로운 변수 생성
    df['EXT_SOURCES_PROD'] = df['EXT_SOURCE_1'] * df['EXT_SOURCE_2'] * df['EXT_SOURCE_3']
    df['EXT_SOURCES_WEIGHTED'] = df.EXT_SOURCE_1*2 + df.EXT_SOURCE2*1 + df.EXT_SOURCE_3*3
    
    np.warnings.filterwarnings('ignore', r'All-NaN (slice|axis) encountered')
    for function_name in ['min','max','mean','namedian','var']:
        feature_name = 'EXT_SOURCES_{}'.format(function_name.upper())
        df[feature_name] = eval('np.{}'.format(function_name))(
            df[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']],axis=1)
        
        
    # 신용 비율
    df['CREDIT_TO_ANNUITY_RATIO'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']
    df['CREDIT_TO_GOODS_RATIO'] = df['AMT_CREDIT'] / df['AMT_GOODS_PRICE']
    # 소득 비율
    df['ANNUITY_TO_INCOME_RATIO'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
    df['INCOME_TO_EMPLOYED_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_EMPLOYED']
    df['INCOME_TO_BIRTH_RATIO'] = df['AMT_INCOME_TOTAL'] / df['DAYS_BIRTH']
    # 시간 비율
    df['EMPLOYED_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['ID_TO_BIRTH_RATIO'] = df['DAYS_ID_PUBLISH'] / df['DAYS_BIRTH']
    df['CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
    df['CAR_TO_EMPLOYED_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_EMPLOYED']
    df['PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
    
    # 같은 그룹인 applications의 통계량으로 그룹화
    group = ['ORGANIZATION_TYPE', 'NAME_EDUCATION_TYPE', 'OCCUPATION_TYPE', 'AGE_RANGE', 'CODE_GENDER']
    df = do_median(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_MEDIAN')
    df = do_std(df, group, 'EXT_SOURCES_MEAN', 'GROUP_EXT_SOURCES_STD')
    df = do_mean(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_MEAN')
    df = do_std(df, group, 'AMT_INCOME_TOTAL', 'GROUP_INCOME_STD')
    df = do_mean(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_MEAN')
    df = do_std(df, group, 'CREDIT_TO_ANNUITY_RATIO', 'GROUP_CREDIT_TO_ANNUITY_STD')
    df = do_mean(df, group, 'AMT_CREDIT', 'GROUP_CREDIT_MEAN')
    df = do_mean(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_MEAN')
    df = do_std(df, group, 'AMT_ANNUITY', 'GROUP_ANNUITY_STD')
    
    # 범주형 변수 인코드(LabelEncoder)
    df, le_encoded_cols = label_encoder(df,None)
    df = drop_application_columns(df)
    return df

In [7]:
# 순열 변수 중요도에 따른 변수 삭제
def drop_application_columns(df):
    drop_list = [
        'CNT_CHILDREN', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START',
        'FLAG_EMP_PHONE', 'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'FLAG_EMAIL', 'FLAG_PHONE',
        'FLAG_OWN_REALTY', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION',
        'REG_CITY_NOT_WORK_CITY', 'OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE',
        'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_YEAR', 
        'COMMONAREA_MODE', 'NONLIVINGAREA_MODE', 'ELEVATORS_MODE', 'NONLIVINGAREA_AVG',
        'FLOORSMIN_MEDI', 'LANDAREA_MODE', 'NONLIVINGAREA_MEDI', 'LIVINGAPARTMENTS_MODE',
        'FLOORSMIN_AVG', 'LANDAREA_AVG', 'FLOORSMIN_MODE', 'LANDAREA_MEDI',
        'COMMONAREA_MEDI', 'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'BASEMENTAREA_AVG',
        'BASEMENTAREA_MODE', 'NONLIVINGAPARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 
        'LIVINGAPARTMENTS_AVG', 'ELEVATORS_AVG', 'YEARS_BUILD_MEDI', 'ENTRANCES_MODE',
        'NONLIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'LIVINGAPARTMENTS_MEDI',
        'YEARS_BUILD_MODE', 'YEARS_BEGINEXPLUATATION_AVG', 'ELEVATORS_MEDI', 'LIVINGAREA_MEDI',
        'YEARS_BEGINEXPLUATATION_MODE', 'NONLIVINGAPARTMENTS_AVG', 'HOUSETYPE_MODE',
        'FONDKAPREMONT_MODE', 'EMERGENCYSTATE_MODE'
    ]
    # 대부분의 flag document 열 삭제
    for doc_num in [2,4,5,6,7,9,10,11,12,13,14,15,16,17,19,20,21]:
        drop_list.append('FLAG_DOCUMENT_{}'.format(doc_num))
        
    df.drop(drop_list, aixs=1, inplace=True)
    
    return df

In [8]:
# 연령 그룹 레이블(int) 반환  ### age 변수 범주화
def get_age_label(days_birth):
    age_years = -days_birth / 365
    if age_years<27: return 1
    elif age_years<40: return 2
    elif age_years<50: return 3
    elif age_years<65: return 4
    elif age_years<99: return 5
    else: return 0

## Bureau Pipeline

In [9]:
# bureau.csv와 bureau_balance.csv 처리 및 데이터프레임 반환
def get_bureau(path,num_rows=None):
    bureau = pd.read_csv(os.path.join(path, 'bureau.csv'), nrows=num_rows)
    # 신용기간과 신용/계좌 만료일 차이
    bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
    bureau['ENDDATE_ID'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']
    # 신용 대 부채 비율 및 차이
    bureau['DEBT_PERCENTAGE'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_CREDIT_SUM_DEBT']
    bureau['DEBT_CREDIT_DIFF'] = bureau['AMT_CREDIT_SUM'] - bureau['AMT_CREDIT_SUM_DEBT']
    bureau['CREDIT_TO_ANNUITY_RATIO'] = bureau['AMT_CREDIT_SUM'] / bureau['AMT_ANNUITY']
    
    # One-hot encoder
    bureau, categorical_cols = one_hot_encoder(bureau, nan_as_cateogry=False)
    # bureau balance 변수와 조인
    bureau = bureau.merge(get_bureau_balance(path,num_rows), how='left', on='SK_ID_BUREAU')
    # 지급이 늦은 flag 달 (만기일 경과한 달)
    bureau['STATUS_12345'] = 0
    for i in range(1,6):
        bureau['STATUS_12345'] += bureau['STATUS_{}'.format(i)]
        
    # balance의 달 수로 집계하고 bureau와 병합 (대출 길이 집계)
    features = ['AMT_CREDIT_MAX_OVERDUE', 'AMT_CREDIT_SUM_OVERDUE', 'AMT_CREDIT_SUM',
        'AMT_CREDIT_SUM_DEBT', 'DEBT_PERCENTAGE', 'DEBT_CREDIT_DIFF', 'STATUS_0', 'STATUS_12345']
    agg_length = bureau.groupby('MONTHS_BALANCE_SIZE')[feature].mean().reset_index()
    agg_length.rename({feat:'LL_' + feat for feat in features}, axis=1, inplace=True)
    bureau = bureau.merge(agg_length, how='left', on='MONTHS_BALANCE_SIZE')
    del agg_length
    gc.collet()
    
    agg_bureau = group(bureau, 'BUREAU_', BUREAU_AGG)  # 일반 대출 집계
    active = bureau[bureau['CREDIT_ACTIVE_Active']==1]
    agg_bureau = group_and_merge(active, agg_bureau, 'BUREAU_ACTIVE_', BUREAU_ACTIVE_AGG)
    closed = bureau[bureau['CREDIT_ACTIVE_Close']==1]
    agg_bureau = group_and_merge(closed,agg_bureau,'BUREAU_CLOSED_',BUREAU_CLOSED_AGG)
    del active, closed
    gc.collect()
    
    # main loan 유형으로 집계
    for credit_type in ['Consumer credit', 'Credit card', 'Mortgage', 'Car loan', 'Microloan']:
        type_df = bureau[bureau['CREDIT_TYPE_'+credit_type]==1]
        prefix = 'BUREAU_' + credit_type.split(' ')[0].upper() + '_'
        agg_bureau = group_and_merge(type_df,agg_bureau,prefix,BUREAU_LOAN_TYPE_AGG)
        del type_df
        gc.collet()
        
    # 지난 x 달로 집계한 시간
    for time_frame in [6,12]:
        prefix = 'BUREAU_LAST{}M_'.format(time_frame)
        time_frame_df = bureau[bureau['DAYS_CREDIT'] >= -39*time_frame]
        agg_bureau = group_and_merge(time_frame_df,agg_bureau,prefix,BUREAU_TIME_AGG)
        del time_frame_df
        gc.collect()
        
    # 지난 대출금 연체 최대값
    sort_bureau = bureau.sort_values(by=['DAYS_CREDIT'])
    gr = sort_bureau.groupby('SK_ID_CURR')['AMT_CREDIT_MAX_OVERDUE'].last().reset_index()
    gr.rename({'AMT_CREDIT_MAX_OVERDUE': 'BUREAU_LAST_LOAN_MAX_OVERDUE'}, inplace=True)
    agg_bureau = agg_bureau.merge(gr,on='SK_ID_CURR',how='left')
    
    # 총 부채 / 총 신용 비율 및 활성 대출금 부채 / 활성 대출금 신용 비율
    agg_bureau['BUREAU_DEBT_OVER_CREDIT'] = agg_bureau['BUREAU_AMT_CREDIT_SUM_DEBT_SUM'] / agg_bureau['BUREAU_AMT_CREDIT_SUM_SUM']
    agg_bureau['BUREAU_ACTIVE_DEBT_OVER_CREDIT'] = agg_bureau['BUREAU_ACTIVE_AMT_CREDIT_SUM_DEBT_SUM']/agg_bureau['BUREAU_ACTIVE_AMT_CREDIT_SUM_SUM']
    
    return agg_bureau

In [10]:
def get_bureau_balance(path, num_rows=None):
    bb = pd.read_csv(os.path.join(path, 'bureau_balance.csv'), nrows=num_rows)
    bb,categorical_cols = one_hot_encoder(bb,nan_as_category=False)
    
    # 각 범주 비율 계산
    bb_processed = bb.groupby('SK_ID_BUREAU')[categorical_cols].mean().reset_index()
    # 납부기간의 최소, 최대, 개수, 평균 (월)
    agg = {'MONTHS_BALANCE': ['min','max','mean','size']}
    bb_precessed = group_and_merge(bb,bb_precessed,'',agg,'SK_ID_BUREAU')
    del bb
    gc.collect()
    
    return bb_processed

## Previous Pipeline

In [12]:
# previoius_application.csv 처리 및 데이터프레임 반환
def get_previous_applications(path,num_rows=None):
    prev = pd.read_csv(os.path.join(path,'previous_applications.csv'),nrows=num_rows)
    pay = pd.read_csv(os.path.join(path,'installments_payments.csv'),nrows=nume_rows)
    
    # 중요한 범주형 변수 one-hot encoding
    ohe_columns = [
        'NAME_CONTRACT_STATUS', 'NAME_CONTRACT_TYPE', 'CHANNEL_TYPE',
        'NAME_TYPE_SUITE', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
        'NAME_PRODUCT_TYPE', 'NAME_CLIENT_TYPE']
    prev, categorical_cols = one_hot_encoder(prev,ohe_columns,nan_as_category=False)
    
    # feature engineering : 비율과 차이
    prev['APPLICATION_CREDIT_DIFF'] = prev['AMT_APPLICATION'] - prev['AMT_CREDIT']
    prev['APPLICATON_CREDIT_RATIO'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    prev['CREDIT_TO_ANNUITY_RATIO'] = prev['AMT_CREDIT']/prev['AMT_ANNUITY']
    prev['DOWN_PAYMENT_TO_CREDIT'] = prev['AMT_DOWN_PAYMENT'] / prev['AMT_CREDIT']
    
    # previous application에 대한 이자율 (간소화)
    total_payment = prev['AMT_ANNUITY'] * prev['CNT_PAYMENT']
    prev['SIMPLE_INTERESTS'] = (total_payment/prev['AMT_CREIDT']-1) / prev['CNT_PAYMENT']
    
    # Active loans - 승인되고 납부되지 않은 (last_due 365243)
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved']==1]
    active_df = approved[approved['DAYS_LAST_DUE']==365243]
    # active loans에서 이미 지불된 금액 확인 (installments.csv 사용)
    active_pay = pay[pay['SK_ID_PREV'].isin(active_df['SK_ID_PREV'])]
    active_pay_agg = active_pay.groupby('SK_ID_PREV')[['AMT_INSTALMENT', 'AMT_PAYMENT']].sum()
    active_pay_agg.reset_index(inplace=True)
    # Active loans : 지불된 것과 할부된 것의 차이
    active_pay_agg['INSTALMENT_PAYMENT_DIFF'] = active_pay_agg['AMT_INSTALMENT'] - active_pay_agg['AMT_PAYMENT']
    # active_df 병합
    active_df = active_df.merge(active_pay_agg, on='SK_ID_PREV', how='left')
    active_df['REMAINING_DEBT'] = active_df['AMT_CREDIT'] - active_df['AMT_PAYMENT']
    active_df['REPAYMENT_RATIO'] = active_df['AMT_PAYMENT'] / active_df['AMT_CREDIT']
    # Active applications 로 집계
    active_agg_df = group(active_df, 'PREV_ACTIVE_', PREVIOUS_ACTIVE_AGG)
    active_agg_df['TOTAL_REPAYMENT_RATIO'] = active_agg_df['PREV_ACTIVE_AMT_PAYMENT_SUM']/                                             active_agg_df['PREV_ACTIVE_AMT_CREDIT_SUM']
    del active_pay, active_pay_agg, active_df
    gc.collect()
    
    # 365.243값을 nan(결측값)으로 대체
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    
    # days last due difference (scehduled x done)
    prev['DAYS_LAST_DUE_DIFF'] = prev['DAYS_LAST_DUE_1ST_VERSION'] - prev['DAYS_LAST_DUE']
    approved['DAYS_LAST_DUE_DIFF'] = approved['DAYS_LAST_DUE_1ST_VERSION'] - approved['DAYS_LAST_DUE']
    
    # 범주형 변수
    categorical_agg = {key:['mean'] for key in categorical_cols}
    # general aggregations
    agg_prev = group(prev, 'PREV_', {**PREVIOUS_AGG, **categorical_agg})
    # agg_prev에 active loan 병합
    agg_prev = agg_prev.merge(active_agg_df, how='left', on='SK_ID_CURR')
    del active_agg_df
    gc.collect()
    # aggregations for approved and refused loans
    agg_prev = group_and_merge(approved, agg_prev, 'APPROVED_', PREVIOUS_APPROVED_AGG)
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    agg_prev = group_and_merge(refused, agg_prev, 'REFUSED_', PREVIOUS_REFUSED_AGG)
    del approved, refused
    gc.collect()
    # Aggregations for Consumer loans and Cash loans
    for loan_type in ['Consumer loans', 'Cash loans']:
        type_df = prev[prev['NAME_CONTRACT_TYPE_{}'.format(loan_type)] == 1]
        prefix = 'PREV_' + loan_type.split(" ")[0] + '_'
        agg_prev = group_and_merge(type_df, agg_prev, prefix, PREVIOUS_LOAN_TYPE_AGG)
        del type_df
        gc.collect()
        
    # 납부가 늦은 대출금으로 SK_ID_PREV 얻기
    pay['LATE_PAYMENT'] = pay['DAYS_ENTRY_PAYMENT'] - pay['DAYS_INSTALMENT']
    pay['LATE_PAYMENT'] = pay['LATE_PAYMENT'].apply(lambda x: 1 if x > 0 else 0)
    dpd_id = pay[pay['LATE_PAYMENT'] > 0]['SK_ID_PREV'].unique()
    # 납부 늦은 대출금으로 집계
    agg_dpd = group_and_merge(prev[prev['SK_ID_PREV'].isin(dpd_id)], agg_prev,'PREV_LATE_', PREVIOUS_LATE_PAYMENTS_AGG)
    del agg_dpd, dpd_id
    gc.collect()
    # Aggregations for loans in the last x months
    for time_frame in [12, 24]:
        time_frame_df = prev[prev['DAYS_DECISION'] >= -30*time_frame]
        prefix = 'PREV_LAST{}M_'.format(time_frame)
        agg_prev = group_and_merge(time_frame_df, agg_prev, prefix, PREVIOUS_TIME_AGG)
        del time_frame_df; gc.collect()
    del prev
    gc.collect()
    
    return agg_prev

## Pos-Cash Pipeline